In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy.linalg import solve
from utility import orbit, BrusselatorModel
import sys
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import eigs


## Newton orbit

In [ ]:
if __name__ == "__main__":
    param_file = "./brusselator_params_2.in"  # JSON file containing model parameters
    model = BrusselatorModel(param_file)
    print("Loaded parameters:", model.N)

f = model.dydt
Jacf = model.brusselator_jacobian

In [ ]:
# Parameters
# Dx = 0.008
# Dy = 0.004
# L = 0.5130 #1.5 #1*0.5130 #characteristic length
# z_L = 1  
# A,B = 2,5.45   
# N = 50
# Initial condition (e.g., a Gaussian pulse)
z_L = model.z_L
N = model.N
A, B = model.A, model.B
z = np.linspace(0, z_L, N)
perturb = np.sin(np.pi*(z/z_L))

X0 = A + 0.01*perturb
Y0 = B/A + 0.01*perturb

y0 = np.concatenate([X0[1:-1],Y0[1:-1]])

# Create the system
# f, Jacf, s, grad_s,ds_T = brusselator_inner(Dx, Dy, L, A,B,z_L,N)


Max_iter = 100
epsilon = 1e-11
T_0 = model.T_ini

### Initial integration

Integrate sufficiently the equation to find a good starting value for the Newton method
We expect y_0 to be in the periodic orbit. 

In [ ]:
t_eval = np.linspace(0.0,20*T_0, 1000)

sol = solve_ivp(fun=f,t_span=[0.0, 20*T_0],
                t_eval=t_eval, 
                y0=y0, method='RK45', 
                **{"rtol": 1e-7,"atol":1e-9}
                )

y_T = sol.y[:,-1]
y_T.shape

In [ ]:
# Re_sorted,Ye_sorted, Ve, We = orbit_finder.subsp_iter_projec(Ve_0, y_T, T_0, f, Jacf, p0, pe, 2)

In [ ]:
# np.diag(Re_sorted[:p0,:p0])

In [ ]:
# eig[:p0]

In [ ]:
orbit_finder = orbit(f,y_T,T_0, Jacf,2, Max_iter, epsilon)

k, T_by_iter, y_by_iter, Norm_B, Norm_Deltay,monodromy_0, monodromy = orbit_finder.Newton_orbit(f,y_T,T_0, Jacf,2, Max_iter, epsilon)


In [ ]:
p0, pe = 10,3

subspace_iter =2
rho = 0.01
eig, eigvec = np.linalg.eig(monodromy)
mask = np.abs(eig) - 1 >  1e-7
Ve_0 = np.real(eigvec[:p0+pe].T)
Ve, _ = np.linalg.qr(Ve_0)
Ve_0.shape
v0 = eigvec[0]

In [ ]:
k, T_by_iter, y_by_iter, Norm_B, Norm_Deltay, monodromy, converged = orbit_finder.Newton_Picard2(f,y_T,
                         T_0, Ve, p0,pe, rho, Jacf, 3, subspace_iter, epsilon)

In [ ]:
def monodromy_mult(y0, T, f,Jacf, v, method = 1, epsilon = 1e-6):
    """
       M*v Matrix-vector multiplication using 
       difference formula to avoid computing the monodromy matrix.
       Args:
            y0: Starting point;
            T: Time to compute the solution;
            f: The rhs of the Ode
            Jacf: The Jacobian of f(A square matrix of size dim x dim )
            method: Integer. 1(default)for finite difference approximation;
                             2 for variational form approximation;
            epsilon: Tolerance(Default = 1e-6) in the finite difference approach.
    """
    dim = len(y0)
    sol = solve_ivp(fun=f,t_span=[0.0, T],
                        t_eval=[T], 
                        y0=y0, method='RK45', 
                        **{"rtol": 1e-13,"atol":1e-15}
                        )
    if method == 1 : #For efficiency we need to have a control over the time step of the integrator or impose high precision
        phi_0_T = sol.y[:,-1]
        sol1 = solve_ivp(fun=f,t_span=[0.0, T],
                        t_eval=[T], 
                        y0=y0 + epsilon*v, method='RK45', 
                        **{"rtol": 1e-13,"atol":1e-15}
                        ) #However if the solution is likely unstable, this method is questionable.
        phi_v_T = sol1.y[:,-1]

        Mv = (phi_v_T - phi_0_T)/epsilon
        print("norm Mv = ",np.linalg.norm(Mv))
    elif method == 2 :
        def Mv_system(t, Y_Mv):
            # Solving numerically the initial value problem (dMv/dt = (Jacf*Mv, Mv(0) = v)
            dMv_dt = Jacf(t,Y_Mv[:dim]) @ Y_Mv[dim:] 
            return np.concatenate([f(t, Y_Mv[:dim]),dMv_dt])

        y_v0 = np.concatenate([y0, v])
        sol_mv = solve_ivp(fun = Mv_system, y0 = y_v0, t_span=[0.0,T], t_eval=[T],method='RK45', 
                        **{"rtol": 1e-7,"atol":1e-9})
        Mv = sol_mv.y[dim:,-1]
    else :
        print("Error : Unavailable method. method should be 1 or 2.")
        sys.exit(1)
        # return

    return Mv

In [ ]:
def Mv_system(t, Mv):
    # Solving numerically the initial value problem (dMv/dt = (Jacf*Mv, Mv(0) = v) 
    dMv_dt = Jacf(t,sol.y[:,0]) @ Mv  # Roughly, we have to compute the flow ie phi(y0, t ) but for the sake of simplicity I use phi(y0,T)
    return dMv_dt.flatten(order = 'F')

#Mv_system(0, v).shape

In [ ]:
# subsp_iter(Ve_0, y0, T, f, Jacf, p, pe, epsilon)
from scipy.linalg import schur

# Example matrix S
S = np.array([[4, 1],
              [2, 3]])

# Perform the Schur decomposition
T, Q, sdim = schur(S, output='complex',sort='lhp')

eigenvalues,_ = np.linalg.eig(T)

# Sort the eigenvalues in increasing order
sorted_indices = np.argsort(np.abs(eigenvalues))[::-1]  # Sorting by absolute value
T_sorted = T[sorted_indices, :][:, sorted_indices]
Q_sorted = Q[:, sorted_indices]
print(sorted_indices)
# Output the results
print("Original Matrix S:")
print(S)

print("\nSchur Decomposition:")
print("T (Upper Triangular):")
print(T)
print("Q (Unitary Matrix):")
print(Q)

print("\nOrdered Schur Decomposition:")
print("Ordered T:")
print(T_sorted)
print("Ordered Q:")
print(Q_sorted)

print("\nSorted Eigenvalues:")
print(eigenvalues[sorted_indices])
T[[1, 1],:]

In [ ]:
import numpy as np
from scipy.linalg import schur, eigvals
A = np.array([[0, 2, 2], [0, 1, 2], [1, 0, 1]])
eig_A = eigvals(A)
T, Z = schur(A,output='real')
print('Without ordering \n')
print(T)
print("Z = \n", Z)
print("Eigenvalues of T :\n", eigvals(T))

T, Z, sdim = schur(A, output='real', sort=lambda x, y: np.linalg.norm([x,y]) > 1e-15)
print("With ordering \n")
print(T)
print("Eigen values of A \n", eig_A)
print("Eigenvalues of T :\n", eigvals(T))
sdim
print("\n", Z)

In [ ]:
T = T_by_iter[k-1]
y0 = np.array(y_by_iter[k-1])
v = np.ones(96)
v = np.eye(1,96,78)[0]
Mv = monodromy_mult(y0, T, f, Jacf, v, 1,1e-6)

np.linalg.norm(Mv - monodromy@v)

In [ ]:
eig, eigvec = np.linalg.eig(monodromy)
mask = np.abs(eig) - 1 >  1e-7
print("Number of Floquet multipliers outside the unit circle\n",len(eig[mask]))

print("Spectral radius of the Monodromy matrix:\n",np.max(np.abs(eig)))

In [ ]:
# Extract real and imaginary parts of eigenvalues
real_parts = np.real(eig)
imaginary_parts = np.imag(eig)
print(imaginary_parts.shape)
# Create the figure and axis
fig, ax = plt.subplots(figsize=(6, 6))

# Plot the unit circle
theta = np.linspace(0, 2 * np.pi, 1000)
circle_x = np.cos(theta)
circle_y = np.sin(theta)
ax.plot(circle_x, circle_y, 'k--', label='Unit Circle')

# Plot the eigenvalues
ax.scatter(real_parts, imaginary_parts, color='r', label='Eigenvalues')

# Set labels and title
ax.set_xlabel(f'Re($\lambda$)')
ax.set_ylabel(f'Im($\lambda$)')
ax.set_title(f'Eigenvalues of the Monodromy matrix on Complex Plane\n with L = {model.L}')

# Set equal aspect ratio
ax.set_aspect('equal', 'box')

# Add grid, legend, and plot
ax.grid(True)
# ax.legend()

# Show the plot
plt.show()

In [ ]:
np.linalg.norm([0.1,0.5], ord=2)

In [ ]:
print(y_by_iter[k-1][:N-2][23])
print(y_by_iter[k-1][N-2:][23])
print(T_by_iter[k])
k

In [ ]:
Tab = np.asarray(y_by_iter[:k-1])
X = Tab[:,N-2:]
Y = Tab[:,:N-2]


In [ ]:
fig, ax = plt.subplots(2,2,sharex='all')
ax[0,0].plot(np.arange(k-1),X.mean(axis=1),'+-')
ax[0,0].set_ylabel(f"$<X>$")

ax[0,1].plot(np.arange(k-1),Y.mean(axis=1),'+-')
ax[0,1].set_ylabel(f"$<Y>$")
ax[1,0].semilogy(np.arange(k),Norm_Deltay[:k],'x-')
ax[1,0].set_xlabel("Newton iterations")
ax[1,0].set_ylabel(f"$\parallel \Delta X \parallel$")
ax[1,1].semilogy(np.arange(k),Norm_B[:k],'x-')
ax[1,1].set_xlabel("Newton iterations")
ax[1,1].set_ylabel(f"$\parallel \phi(X^*(0),T) - X^*(T) \parallel$")
# ax[1,1].set_ylabel(f"$\parallel (r,s) \parallel_2$")
fig.set_size_inches((8,8))
fig.suptitle(f'Brusselator model: $L=%.4f$ \n $T = %.4f$ ' % (model.L, T_by_iter[k-1]))
fig.subplots_adjust(left=0.09, bottom=0.1, right=0.95, top=0.90, hspace=0.35, wspace=0.55)
# plt.savefig(f'./Results/Modulated_Laser_T_known_m_{str(m)}.png')

In [ ]:
y0 = np.array(y_by_iter[k-1])
t_eval = np.linspace(0.0,2*T_by_iter[k-1], 1000)

sol = solve_ivp(fun=f,t_span=[0.0, 2*T_by_iter[k-1]],
                t_eval=t_eval, 
                y0=y0, method='RK45', 
                **{"rtol": 1e-7,"atol":1e-9}
                )

y_T = sol.y[:,-1]
y_T.shape

In [ ]:
Xmean = sol.y[:N-2,:]
#Xmean = np.mean(Xmean,axis = 0)
Ymean = sol.y[N-2:,:]
#Ymean = np.mean(Ymean,axis = 0)

In [ ]:
fig, ax = plt.subplots(1,2)
ax[0].plot(t_eval, Xmean[23], label = '<X>')
ax[0].plot(t_eval, Ymean[23], label='<Y>')
ax[0].legend()
ax[0].set_ylabel(f"$Concentrations$")
ax[0].set_xlabel("t")
ax[1].plot(Xmean[23],Ymean[23])
ax[1].set_ylabel(f"$<Y>$")
ax[1].set_xlabel(f"$<X>$")
fig.set_size_inches((10,5))
fig.suptitle(f'Brusselator Model with Dirichlet BCs:')
fig.subplots_adjust(left=0.09, bottom=0.1, right=0.95, top=0.90, hspace=0.35, wspace=0.55)
# plt.savefig(f'./Results/brusselator_1D.png')

In [ ]:
import numpy as np

from scipy.sparse.linalg import LinearOperator

def mv(v,y):

    return np.array([2*v[0]*y, 3*v[1], v[2], v[3]])

y = 1 
A = LinearOperator((4,4), matvec=lambda v : mv(v,y))

print(A)

print(A.matvec(np.ones(4)))

A @ np.ones(4)

In [ ]:
from scipy.sparse.linalg import eigs
# eigenval, Vp = eigs(A, k=2, which = 'LM', v0 =  0.5*np.ones(4))

In [ ]:
eigenval
Vp

M = Vp@Vp.T

Q = np.eye(4,4)-Vp@Vp.T
Q.shape

In [ ]:
def base_Vp(v0, y0, T, f, Jacf, p, epsilon):

    #Ce que je veux c'est une base du souspace dominant de M.
    dim = len(y0)
    Mv = LinearOperator((dim,dim),matvec = lambda v : monodromy_mult(y0, T, f,Jacf, v, method = 2, epsilon = 1e-6))
    
    eigenval, Vp = eigs(Mv, k=p, which = 'LM', v0 = v0)
    return eigenval, Vp




eigenval, Vp = base_Vp(v0, y0, T, f, Jacf, 3, epsilon)


In [ ]:
print(eig[mask])
np.abs(eigenval - eig[mask])

In [ ]:
def brusselator_inner(Dx, Dy,L,A,B,z_L,N):

    # Discretize space
    h = z_L / (N - 1)  # Grid spacing
    def Lap_mat(N):
        main_diag = -2  * np.ones(N)
        off_diag = np.ones(N - 1)
        laplacian = np.diag(main_diag) + np.diag(off_diag, k=1) + np.diag(off_diag, k=-1)
        return laplacian
    
    def dydt(t, y):
        X = y[:N-2]
        Y = y[N-2:] 
        
        # Central difference for ALL INNER POINTS (indices 1 to N-2)
        # Inner points (central difference)
       
        # Dirichlet BCs
        X_BCs = A*np.eye(1,N-2,0)[0] + A*np.eye(1,N-2,N-3)[0]
        Y_BCs = (B/A)*np.eye(1,N-2,0)[0] + (B/A)*np.eye(1,N-2,N-3)[0]
        
        d2Xdz2 = (1/h**2)*(Lap_mat(N-2)@X + X_BCs)
        d2Ydz2 = (1/h**2)* (Lap_mat(N-2)@Y + Y_BCs)
        # Reaction-diffusion equation
        dXdt = Dx/(L**2) * d2Xdz2 + Y*(X**2) - (B+1)*X + A
        dYdt = Dy/(L**2) * d2Ydz2 - Y*(X**2) + B*X

        dydt = np.concatenate([dXdt, dYdt])
        return dydt
    

    def brusselator_jacobian(t,y): # A faire : Stockage creuse de la matrice 
        X = y[:N-2]
        Y = y[N-2:]
        n = len(X) #The inner points of the mesh
        h = z_L / (N - 1)
        
        # Diffusion coefficients
        alpha_x = Dx / (L*h)**2
        alpha_y = Dy / (L*h)**2
        
        # Fill J_y_by_iter and J_YY (tridiagonal blocks)
        Jxx = alpha_x*Lap_mat(n) - (B+1)*np.eye(n) + 2*np.diag(X*Y)
        Jyy = alpha_y*Lap_mat(n) - np.diag(X**2)
        Jyx = np.diag(X**2)
        Jxy = B*np.eye(n) - 2*np.diag(X*Y)
        #Assembling the whole matrix
        top = np.hstack((Jxx, Jyx))  # Horizontal stacking of A11=M-I and A12=b
        bottom = np.hstack((Jxy,Jyy)) # Horizontal stacking of A21=c and A22=d
        J = np.vstack((top, bottom))  # Vertical stacking of the two rows
        return J
    #Phase conditions

    def s1(t,y): 
        
        return dydt(t,y)[0]
    
    def ds1_dy(t,y):
        return brusselator_jacobian(t,y)[0,:]
    def ds1_dT(t,y):

        return 0.0
    def s2(t,y,y_preced):
        return (y-y_preced)@dydt(t,y_preced)
    def ds2_dy(t,y):
        return dydt(t,y)
    return dydt, brusselator_jacobian, s1, ds1_dy, ds1_dT


In [ ]:
## Trial Run
# Example usage
if __name__ == "__main__":
    # Parameters
    Dx = 0.008
    Dy = 0.004
    L = 0.5130 #1.5 #1*0.5130 #characteristic length
    z_L = 1  #Domain length
    A,B = 2,5.45   
    N = 50 # Number of grid points
 
    
    # Initial condition (e.g., a Gaussian pulse)
    z = np.linspace(0, z_L, N)
    # perturb = 0.1 * np.exp(-((z - z_L / 2)**2)) #*np.concatenate([A*np.ones(N),(B/A)*np.ones(N)])
    perturb = np.sin(np.pi*(z/z_L))
    X0 = A + 0.1*perturb
    Y0 = B/A + 0.1*perturb
    # # Enforce Dirichlet BCs on the initial condition
    # Y0[0], Y0[-1] = B/A, B/A  
    # X0[0], X0[-1] = A, A
    y0 = np.concatenate([X0[1:-1],Y0[1:-1]])

    # Create the system
    # system = brusselator_1D(Dx, Dy, L, A,B,z_L,N)
    f, Jacf, s, grad_s,ds_T = brusselator_inner(Dx, Dy, L, A,B,z_L,N)
    tf = 200
    t_span = (0, tf)  # Time span
    sol = solve_ivp(f, t_span, y0, method='BDF', t_eval=np.linspace(0, tf, 1000), jac = Jacf)
y_ini = sol.y[:,-1]
y_ini.shape
# Plot the solution
plt.figure(figsize=(10, 6))
for i, t in enumerate(sol.t):
    if i % 900== 0:  # Plot every 50th time step
        plt.plot(z[1:-1], sol.y[:N-2, i],'+', label=f"X, t={t:.2f}")
        plt.plot(z[1:-1], sol.y[N-2:, i],'+', label=f"Y, t={t:.2f}")

plt.xlabel("z")
plt.ylabel("Concentrations X, Y")
plt.legend()
plt.title("Brusselator Model with Dirichlet BCs")
plt.show()

#Heat map 
Xmean = sol.y[:N-2,:]
Xmean = np.mean(Xmean,axis = 0)
Ymean = sol.y[N-2:,:]
Ymean = np.mean(Ymean,axis = 0)

fig, ax = plt.subplots(1,2)
ax[0].plot(np.linspace(0, tf, 1000), Xmean, label = '<X>')
ax[0].plot(np.linspace(0, tf, 1000), Ymean, label='<Y>')
ax[0].legend()
ax[0].set_ylabel(f"$Concentrations$")
ax[0].set_xlabel("t")
ax[1].plot(Xmean,Ymean)
ax[1].set_ylabel(f"$<Y>$")
ax[1].set_xlabel(f"$<X>$")
fig.set_size_inches((10,5))
fig.suptitle(f'Brusselator Model with Dirichlet BCs:')
fig.subplots_adjust(left=0.09, bottom=0.1, right=0.95, top=0.90, hspace=0.35, wspace=0.55)
# plt.savefig(f'./Results/brusselator_1D.png')

In [ ]:
mu = np.array([5.1, 3.8, 2.5, 0.9, 0.85, 0.6])
mod_mu = np.abs(mu)
rho = .6
p = np.sum(mod_mu > rho)

print("Indice p:", p)
# Output: Indice p: 3


In [ ]:
np.ones((5,5))[:3,:3]
def p():
    return "Failure"

p()